In [ ]:
%autosave 0

In [ ]:
# Importing Libraries
import json
import csv
import re
import nltk
import numpy as np
import pandas as pd
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from wordcloud import WordCloud
import matplotlib.pyplot as plt

/Users/hilinasisay/opt/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [ ]:
# Read JSON File
csv_root = '''/Users/scl/Python/final/d1/'''
json_name = 'image_review_all.json'
list_of_dict = []
for line in open(csv_root+json_name, 'r'):
    list_of_dict.append(json.loads(line))

In [ ]:
# Define a List to Store the Relationship Between Text Evaluations and Ratings, Including User ID and Restaurant ID
final_res_list = []
#list_of_dict

In [ ]:
# Initialize User and Restaurant Index Counters
cur_user_index = 0
cur_rest_index = 0

# Create an Empty List to Store the Final Results
user_map = {}
rest_map = {}

# Define Functions to Handle User and Restaurant IDs, and Process the Data From the JSON File
def handle_user_id(user_id_str):
    if user_id_str in user_map:
        return user_map[user_id_str]
    global cur_user_index
    user_map[user_id_str] = cur_user_index
    cur_user_index += 1
    return user_map[user_id_str]

def handle_rest_id(rest_id_str):
    if rest_id_str in rest_map:
        return rest_map[rest_id_str]
    global cur_rest_index
    rest_map[rest_id_str] = cur_rest_index
    cur_rest_index += 1
    return rest_map[rest_id_str]

In [ ]:
# Iterate Through Each Element in the List of Dictionaries From the JSON File
for element in list_of_dict:
    # Extract User and Restaurant IDs
    user_id_str = element['user_id']
    rest_id_str = element['business_id']
    # Process User and Restaurant IDs Using the Defined Functions
    user_id = handle_user_id(user_id_str)
    rest_id = handle_rest_id(rest_id_str)
    # Extract the Review Text and Rating
    text = element['review_text']
    rating = element['rating']
    # Create a Tuple Containing User ID, Review Text, Restaurant ID, and Rating
    # Add the Tuple to the Final Results List
    final_res_list.append((user_id, text, rest_id, rating))

In [ ]:
# Print Length and First 10 Elements of Text-Rating Pairs List
print(len(final_res_list))
print(final_res_list[:10])

In [ ]:
# Create New CSV File to Store Text-Rating Pairs
csv_file = 'text_rating_pairs_huge_with_id.csv'
with open(csv_root+csv_file, 'w', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerow(['user_id','text','rest_id', 'rating'])
    for user_id, text, rest_id, rating in final_res_list:
        writer.writerow([user_id, text, rest_id, rating])

In [ ]:
# Define Text Preprocessing Function
def text_preprocessing(text):
    # Remove Punctuation
    text = re.sub(r'[^\w\s]', '', text)
    # Remove URL
    text = re.sub(r'http\S+', '', text)
    # Remove Stop Words
    stop_words = stopwords.words('english')
    text = ' '.join([word for word in text.split() if word not in stop_words])
    # Convert to Lowercase
    text = text.lower()
    #  Tokenize Text
    tokens = nltk.word_tokenize(text)
    # Apply Stemming
    stemmer = PorterStemmer()
    tokens = [stemmer.stem(token) for token in tokens]
    # Apply Lemmatization
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token) for token in tokens]
    # Return Processed Text
    return ' '.join(tokens)

In [ ]:
# Test Text Preprocessing Function
text = 'This is a great restaurant with delicious food and friendly service. Highly recommended! https://www.google.com'
print(text_preprocessing(text))

In [ ]:
# Define Preprocessing Function to Apply on DataFrame Rows
def preprocess(row):
    text = str(row['text'])
    text_pp = text_preprocessing(text)
    row['text_pp'] = text_pp
    return row

In [ ]:
# Read Original CSV and Apply Preprocessing Function
csv_df = pd.read_csv(csv_root+csv_file)
csv_df

In [ ]:
# Add Empty 'text_pp' Column and Apply Preprocessing Function to DataFrame
csv_df['text_pp'] = ''
pp_df = csv_df.apply(preprocess, axis=1)
pp_df

In [ ]:
'''
The dataset is split into three parts: training, test, and validation sets, with an 8:1:1 ratio. 
First, we divide the dataset into a training set (80%) and a test set (20%). 
Then, we further split the test set into two equal parts to create the test set and validation set (each 10%). 
It is important to ensure that each rating value is equally distributed across the subsets.
'''
train_df, test_df = train_test_split(pp_df, test_size=0.2, stratify=pp_df['rating'])
test_df, val_df = train_test_split(test_df, test_size=0.5, stratify=test_df['rating'])

# Save Training, Test, and Validation Sets as CSV Files
columns_to_save = ['user_id', 'text_pp', 'rest_id', 'rating']
train_df[columns_to_save].to_csv(csv_root+'train_huge_with_id.csv', index=False)
test_df[columns_to_save].to_csv(csv_root+'test_huge_with_id.csv', index=False)
val_df[columns_to_save].to_csv(csv_root+'valid_huge_with_id.csv', index=False)

In [ ]:
# Read CSV
train_csv = 'train_huge_with_id.csv'
valid_csv = 'valid_huge_with_id.csv'
test_csv = 'test_huge_with_id.csv'
train_df = pd.read_csv(csv_root+train_csv)
valid_df = pd.read_csv(csv_root+valid_csv)
test_df = pd.read_csv(csv_root+test_csv)
print(train_df.head(3))
print(valid_df.head(3))
print(test_df.head(3))

In [ ]:
full_df = pd.concat([train_df, valid_df, test_df])
full_df

In [ ]:
# Define a OneHotEncoder for processing User IDs
one_hot_user = OneHotEncoder(handle_unknown='ignore')
# Define a OneHotEncoder for processing Restaurant IDs
one_hot_rest = OneHotEncoder(handle_unknown='ignore')

In [ ]:
# Apply the TF-IDF Vectorizer to Transform the Text Into a Vector Representation
# Set the Maximum Number of Features to 100000, the Minimum Document Frequency to 3, and Use 1-3 Grams
vectorizer = TfidfVectorizer(max_features=100000, min_df=3, ngram_range=(1,3))
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

In [ ]:
# Define the preprocessor that combines the three encoders
preprocessor = ColumnTransformer(
    transformers=[
        ('user', one_hot_user, ['user_id']),
        ('rest', one_hot_rest, ['rest_id']),
        ('text', vectorizer, 'text_pp')])

In [ ]:
# Define a logistic regression model
lr = LogisticRegression(C=1.0, penalty="l1", class_weight='balanced', max_iter=100, solver='liblinear')

In [ ]:
# Create the Overall Model by Connecting the Preprocessor and Logistic Regression Using a Pipeline
model = Pipeline(steps=[('preprocessor', preprocessor),('classifier', lr)])

In [ ]:
# Train the model
train_df['text_pp'] = train_df['text_pp'].fillna('')
model.fit(train_df[['user_id', 'rest_id', 'text_pp']], train_df['rating'])

In [ ]:
# Test the model
test_df['text_pp'] = test_df['text_pp'].fillna('')
# Predict Ratings
y_pred = model.predict(test_df[['user_id', 'rest_id', 'text_pp']])
# Calculate the Accuracy
accuracy = accuracy_score(test_df['rating'], y_pred)
print(f'Accuracy: {accuracy:.2f}')

In [ ]:
# Evaluate the Model's Performance Using a Confusion Matrix and Calculate Accuracy
# Print the Confusion Matrix and Accuracy
y_test = test_df['rating']
cm = confusion_matrix(y_test, y_pred)
print("Confusion matrix:")
print(cm)
accuracy = np.sum(np.diag(cm)) / np.sum(cm)
print("Accuracy: {:.2f}%".format(accuracy * 100))

In [ ]:
# Generate and Print the Classification Report for the Model's Performance
report = classification_report(y_test, y_pred)
print(report)

In [ ]:
'''
The final accuracy has increased by one percentage point. 
The f1 scores corresponding to all rating categories, except for the rating of 2, have increased as well. 
The experimental results show that incorporating user and restaurant information has improved the model's performance, 
making the entire system more accurate.
'''

In [ ]:
# Conduct a Live Test With a Sample Review Text: "Great Food and Service. Highly Recommended."
new_text = "Great food and service. Highly recommended."
new_text_pp = text_preprocessing(new_text)
print(new_text_pp)

# Transform the Preprocessed Text Into a TF-IDF Vector
new_vector = vectorizer.transform([new_text_pp])
print(new_vector)

# Predict the Rating for the New Text
new_pred = model.predict(new_vector)
print(new_pred)

# Print the Prediction Result
print("The predicted rating for the new text is:", new_pred[0])

In [ ]:
# Now we will analyze the weight-feature word relationship.
# Examine the model's weights, which form a matrix with a shape of (number of categories * number of features)
model.coef_

In [ ]:
# The second feature appears to have a significant downward trend; 
# Let's determine which feature corresponds to it
feature_names = vectorizer.get_feature_names()
print(feature_names[2])

In [ ]:
# Find all indices with significant weights
indices = np.where(np.all(np.diff(model.coef_, axis=0) < 0, axis=0))[0]

In [ ]:
# Print out the names of these features
selected_feature_names = np.take(feature_names, indices)
selected_feature_names

In [ ]:
'''
We can see that some of these words have strong emotional connotations
For instance, undercooked, tasteless, worst, cold, frozen, and burned are very negative comments about a restaurant.
'''

In [ ]:
# Word Cloud Analysis
train_csv = 'train.csv'
df = pd.read_csv(csv_root+train_csv)

# Filter out reviews with ratings of 1 and 4-5
low_ratings = df[(df['rating'] <= 1)]
high_ratings = df[(df['rating'] >= 4) & (df['rating'] <= 5)]

# Merge the review text into one string
low_text = ' '.join(low_ratings['text_pp'].astype(str))
high_text = ' '.join(high_ratings['text_pp'].astype(str))

# Generate Word Clouds
low_wc = WordCloud().generate(low_text)
high_wc = WordCloud().generate(high_text)

In [ ]:
# Display the word cloud for negative reviews
'''
We can see that there are many negative comments about the burger.
People often complain that they ordered something but got another thing, expressing dissatisfaction with the result.
'''
plt.imshow(low_wc, interpolation='bilinear')
plt.axis('off')
plt.show()

In [ ]:
# Display the word cloud for positive reviews
plt.imshow(high_wc, interpolation='bilinear')
plt.axis('off')
plt.show()